## 文本分类任务实战

- 数据集构建：影评数据集进行情感分析（分类任务）
- 词向量模型：加载训练好的词向量或者自己训练都可以
- 序列网络模型：训练RNN模型进行识别

![title](./img/SentimentAnalysis2.png)

### RNN模型所需数据解读：

![title](./img/1.png)

In [1]:
import os
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
import numpy as np
import time

import logging

from collections import Counter
from pathlib import Path
# from tqdm import tqdm

### 加载影评数据集

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data()

In [3]:
# 训练集和测试集各有25000条影评，已经经过分词处理
x_train.shape, x_test.shape

((25000,), (25000,))

In [4]:
# 每个元素都是一个词的id
# 如果想通过id找到词，需要使用get_word_index()方法拿到word2id
print(x_train[0])

# 该数据集中每一条数据都是以id=1开头，id为1表示句子的起始符

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [5]:
# 注意这里得到的是word2id，如果想要得到id2word，简单的转换一下即可
word2id = tf.keras.datasets.imdb.get_word_index()

In [6]:
# 可以查看一下词频最高的几个词是什么
sorted(list(word2id.items()), key=lambda x:x[1])[:10]

[('the', 1),
 ('and', 2),
 ('a', 3),
 ('of', 4),
 ('to', 5),
 ('is', 6),
 ('br', 7),
 ('in', 8),
 ('it', 9),
 ('i', 10)]

注意返回的x_train中单词的id和get_word_index()方法返回的单词的id不是一一对应的，如果index_from参数设置为3，那么实际上x_train中的id是一个词原来的id再加上3，例如get_word_index()方法返回的字典中the的id是1，那么在x_train中id为4的词才对应的是the，而不是1, 也就是说在x_train中的id从4开始才是实际的单词，id为0,1,2,3都是表示一些特殊的标记，而不是实际的单词

imdb.load_data()方法中默认使用1表示开始字符，2表示oov字符，即如果文本中出现了不在字典中的单词，会使用2代替，但是这些字符不会出现在get_word_index()方法返回的dict中，因此需要对get_word_index()方法返回的dict先进行一些处理

### 现在制作新的词ID映射表，空出来4个位置的目的是加上特殊字符

In [7]:
# i+3是为了留出0,1,2,3这4个空位，用来保存一些特殊用处的字符
# 加3是因为imdb.load_data()设置了index_from参数为3，
# 如果修改了index_from为N，那么应该加N
# 原来第一个词the的index是1，现在变成4
word2id_new = {w: i + 3 for w, i in word2id.items()}

# <pad>表示补齐字符，如果一些文章的长度不足，使用<pad>补齐
word2id_new['<pad>'] = 0

# <start>表示文章的起始字符
word2id_new['<start>'] = 1

# <unk>表示未出现在词典中的词，即oov(out-of-vocabulary)
word2id_new['<unk>'] = 2

# id为3的位置空出来，暂时不使用

sorted(list(word2id_new.items()), key=lambda x:x[1])[:10]

[('<pad>', 0),
 ('<start>', 1),
 ('<unk>', 2),
 ('the', 4),
 ('and', 5),
 ('a', 6),
 ('of', 7),
 ('to', 8),
 ('is', 9),
 ('br', 10)]

In [8]:
# 交换一下就得到id2word
id2word_new = {i: w for w, i in word2id_new.items()}

id2word_new[0], id2word_new[4]

('<pad>', 'the')

In [9]:
# 按文本长度进行排序
def sort_by_len(x, y):
    
    # 这行代码很精妙，仔细理解一下
    idx = sorted(range(len(x)), key=lambda i: len(x[i]))
    
    # 拿到排序的id后，传回原来的数据中，得到排好序的数据
    return x[idx], y[idx]

# 现在得到整理后的数据集
x_train, y_train = sort_by_len(x_train, y_train)
x_test, y_test = sort_by_len(x_test, y_test)

In [10]:
x_train[:10]

array([list([1, 13, 586, 851, 14, 31, 60, 23, 2863, 2364, 314]),
       list([1, 14, 20, 9, 394, 21, 12, 47, 49, 52, 302]),
       list([1, 1390, 128, 2257, 723, 8965, 60, 48, 25, 28, 296, 12]),
       list([1, 12039, 4, 12632, 127, 6, 117, 67102, 5, 6, 20, 91, 3939]),
       list([1, 6741, 20576, 9, 321, 9, 14, 22, 29, 166, 6, 1429, 255]),
       list([1, 196, 357, 16445, 115, 28, 13, 77, 38, 1264, 8, 67, 277, 898, 1686]),
       list([1, 11300, 390, 1351, 9, 4, 118, 390, 7, 11300, 45, 61, 514, 390, 7, 11300]),
       list([1, 11300, 390, 1351, 9, 4, 118, 390, 7, 11300, 45, 61, 514, 390, 7, 11300]),
       list([1, 931, 14, 20, 9, 1167, 9, 394, 55, 6415, 78, 2956, 963, 458, 24, 168]),
       list([1, 57, 931, 379, 20, 116, 856, 42, 433, 881, 57, 281, 33, 32, 1771, 12])],
      dtype=object)

In [11]:
# 把id序列转化为相应的字符串
def decode_review(text):
    return ' '.join([id2word_new[i] for i in text])

# 显示其中一个评价
decode_review(x_train[0])

"<start> i wouldn't rent this one even on dollar rental night"

上面的id2word_new不用于训练模型，只是为了加深对该数据集的理解，并且能将原来的id列表转换成原来的句子

## 下面开始构建用于训练的词典

In [12]:
# 将中间结果保存到本地，万一程序崩了还得重玩，保存的是文本数据，不是ID
def write_file(f_path, xs, ys):
    with open(f_path, 'w', encoding='utf-8') as f:
        for x, y in zip(xs, ys):
            
            # 将id转换成对应的单词，然后将字符串写入到硬盘
            # 每一行的第一个数字是标签值，然后加一个\t字符
            # 后跟每一条影评的句子(切片操作从1开始是为了去掉开始字符)
            # 现在每一条在train.txt和test.txt中的数据如下所示：
            # 0	you'd better choose paul verhoeven's even if you have watched it
            # 0	ming the merciless does a little bardwork and a movie most foul
            f.write(str(y)+ '\t' +' '.join([id2word_new[i] for i in x][1:]) + '\n')

write_file('./data/train.txt', x_train, y_train)
write_file('./data/test.txt', x_test, y_test)

### 构建语料表，基于词频来进行统计

In [13]:
counter = Counter()

with open('./data/train.txt', encoding='utf-8') as f:
    text_list = f.readlines()
    for line in text_list:
        line = line.strip()
        label, words = line.split('\t')
        words = words.split(' ')
        
        # Counter的update方法可以是序列类型，
        # 如果要更新的关键字已存在，则对它的值进行求和；如果不存在，则添加
        counter.update(words)

# most_common()方法如果不传参数表示考虑所有的元素
counter.most_common(10)

[('the', 336148),
 ('and', 164097),
 ('a', 163040),
 ('of', 145847),
 ('to', 135708),
 ('is', 107313),
 ('br', 101871),
 ('in', 93934),
 ('it', 79060),
 ('i', 77142)]

In [14]:
# 只考虑频率大于10的词
words = ['<pad>'] + [w for w, freq in counter.most_common() if freq >= 10]

# 最后剩下20598个词
len(words)

20598

In [15]:
Path('./vocab').mkdir(exist_ok=True)

# 保存到硬盘
with open('./vocab/word.txt', 'w', encoding='utf-8') as f:
    for w in words: f.write(w + '\n')

## 现在得到新的word2id映射表，和上面的word2id是不一样的

In [16]:
word2idx = dict()

# 读入刚才制作的映射表
with open('./vocab/word.txt', encoding='utf-8') as f:
    for i, line in enumerate(f):
        line = line.strip()
        word2idx[line] = i
        
list(word2idx.items())[:10]

[('<pad>', 0),
 ('the', 1),
 ('and', 2),
 ('a', 3),
 ('of', 4),
 ('to', 5),
 ('is', 6),
 ('br', 7),
 ('in', 8),
 ('it', 9)]

## embedding层
- 可以基于网络来训练，也可以直接加载别人训练好的，一般都是加载预训练模型
- 这里有一些常用的：https://nlp.stanford.edu/projects/glove/

![title](./img/SentimentAnalysis3.png)

In [17]:
# embedding是所有词的词向量表，每一行表示一个词向量
# 里面有20598个不同的词，再加一个unknown，20599*50
# 初始化一个全0矩阵，+1表示第20598行对应的向量表示unknown
embedding = np.zeros((len(word2idx) + 1, 50)) 

# 使用别人已经预训练好的词向量，读入文件glove.6B.50d.txt，
# 里面包含40万个词的词向量，每个词向量的长度是50
with open('./data/glove.6B.50d.txt', encoding='utf-8') as f:
    
    # count统计在词典中的词的数量
    count = 0
    for i, line in enumerate(f):
        if i % 100000 == 0: print('- At line {}'.format(i)) #打印处理了多少数据
        
        # 每一行文本的格式如下所示
        # the 0.418 0.24968 -0.41242 0.1217 0.34527 -0.044457 -0.49688...
        line = line.strip()
        sp = line.split(' ')
        
        # vec表示该词的向量
        word, vec = sp[0], sp[1:]
        
        # 如果一个词在我们定义的词典中，就将对应下标的向量转换成已经训练好的向量
        if word in word2idx:
            count += 1
            embedding[word2idx[word]] = np.array(vec, dtype='float32') # 将词转换成对应的向量

- At line 0
- At line 100000
- At line 200000
- At line 300000


现在已经得到每个词索引所对应的向量

In [18]:
print("[%d / %d] words have found pre-trained values"%(count, len(word2idx)))

# embedding是所有词的词向量表，每一行表示一个词向量
# 保存得到的词向量矩阵，避免重复处理数据
np.save('./vocab/word.npy', embedding)
print('Saved ./vocab/word.npy')

[19676 / 20598] words have found pre-trained values
Saved ./vocab/word.npy


### 构建训练数据

- 注意所有的输入样本必须都是相同shape(文本长度，词向量维度等)，如果一篇文档过长就将其截断，过短则补齐

### 数据生成器
- tf.data.Dataset.from_tensor_slices(tensor)：将tensor沿其第一个维度切片，返回一个含有N个样本的数据集，这样做的问题就是需要将整个数据集整体传入，然后切片建立数据集类对象，比较占内存。

- tf.data.Dataset.from_generator(data_generator,output_data_type,output_data_shape)：从一个生成器中不断读取样本

In [19]:
# 自定义生成器函数
def data_generator(f_path, params):
    
    with open(f_path, encoding='utf-8') as f:
        
        print('Reading', f_path)
        
        for line in f:
            
            # 每一行文本的格式如下：
            # 0 this movie is terrible but it has some good effects
            # 第一个数字是标签值，后接一个\t字符
                
            line = line.strip()
            label, text = line.split('\t')
            text = text.split(' ')
            
            # 将句子中的每个词转换成id
            # params是随后定义的一个字典，保存了所有的参数
            # 如果词不在字典中，则用len(word2idx)代替(即unknown对应的id)
            x = [params['word2idx'].get(w, len(word2idx)) for w in text]
            
            # params['max_len']指定了句子的最大长度，不足则补齐，超过则截断
            if len(x) >= params['max_len']: x = x[:params['max_len']] # 截断操作
            else: x += [0] * (params['max_len'] - len(x))  # 补齐操作
                
            y = int(label)
            
            # yield关键字的用法请看其他笔记，简单来说就是当程序执行到yield关键字时，
            # 会将x，y返回，这里就可以看成是返回以一个样本数据，x是特征，y是标签
            
            # 当下一次调用data_generator()函数时，函数不会重头开始执行，而是从上一次
            # 遇到yield关键字的地方开始执行，这里也就是回到for循环的开始位置
            yield x, y

In [20]:
# is_training参数表示是从训练集中取数据还是测试集中取数据
def dataset(is_training, params):
    
    # ()表示是一个标量，也就是一个数字
    _shapes = ((params['max_len'],), ())
    _types = (tf.int32, tf.int32)
      
    # 参数output_shapes和output_types的类型分别是
    # tf.TensorShape和tf.dtypes.DType
    # 用于指定张量的形状和类型，data_generator()函数返回两个值，
    # 因此_shapes和_types需要有两个元素，分别对应形状和类型
    
    # 从训练集取数据
    if is_training:
        # from_generator()方法返回一个dataset对象
        # 注意这里的lambda函数没有输入参数，所以直接跟冒号
        ds = tf.data.Dataset.from_generator(
            lambda : data_generator(params['train_path'], params),
            output_shapes = _shapes,
            output_types = _types,)
        
        ds = ds.shuffle(params['num_samples'])
        ds = ds.batch(params['batch_size'])
        
        # 设置缓存序列，根据可用的CPU动态设置并行调用的数量，说白了就是加速
        ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    # 从测试集取数据
    else:
        ds = tf.data.Dataset.from_generator(
            lambda : data_generator(params['test_path'], params),
            output_shapes = _shapes,
            output_types = _types,)
        ds = ds.batch(params['batch_size'])
        ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  
    return ds

### 自定义网络模型
- 定义好都有哪些层
- 前向传播走一遍就行了

embedding_lookup的作用：

![title](./img/SentimentAnalysis5.png)

![title](./img/SentimentAnalysis16.png)

![title](./img/2.png)

In [21]:
# 自定义模型类，继承tf.keras.Model
# 必须重写__init__方法和call()方法
class Model(tf.keras.Model):
    
    def __init__(self, params):
        
        super().__init__()
        
        # 下面这些都是类的属性，到时候可以直接通过属性调用
        
        # 将词向量矩阵构建成Variable类型
        # embedding是所有词的词向量表，每一行表示一个词向量
        self.embedding = tf.Variable(np.load('./vocab/word.npy'),
                                     dtype=tf.float32,
                                     name='pretrained_embedding',
                                     trainable=False)
        
        self.drop1 = tf.keras.layers.Dropout(params['dropout_rate'])
        self.drop2 = tf.keras.layers.Dropout(params['dropout_rate'])
        self.drop3 = tf.keras.layers.Dropout(params['dropout_rate'])
        
        # 将LSTM包装在Bidirectional层中
        # 现在相当于每层RNN有两层LSTM，一层正向，一层反向
        # 正向层和反向层不共享参数
        
        # 第一个参数表示通过LSTM层后得到的特征个数(实际上是sigmoid层神经元的个数)
        # 这个参数决定了Cell_state和hidden_state向量的维数
        
        # return_sequences表示是否返回所有序列，
        # 如果为Flase则只返回最后 一个时刻的输出值，默认为Flase
        self.rnn1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
            params['rnn_units'], return_sequences=True))
        
        self.rnn2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
            params['rnn_units'], return_sequences=True))
        
        # 最后一层rnn只返回最后一个时刻的输出值
        self.rnn3 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
            params['rnn_units'], return_sequences=False))

        self.drop_fc = tf.keras.layers.Dropout(params['dropout_rate'])
        
        # 由于是双向循环网络，前向层和后向层的特征会拼接在一起，因此全连接层
        # 神经元的个数应该是两倍
        self.fc = tf.keras.layers.Dense(2*params['rnn_units'], activation="relu")

        self.out_linear = tf.keras.layers.Dense(2, activation="softmax")

  
    def call(self, inputs, training=False):
        
        # 如果张量的元素类型不是整型，转换成整型
        if inputs.dtype != tf.int32: inputs = tf.cast(inputs, tf.int32)
        
        # 在embedding中查找词向量，后续的代码中传进来的inputs就是一个句子，
        # 句子中的词都用id表示，例如：[1, 2, 3, 4, 5, ..., 100]
        x = tf.nn.embedding_lookup(self.embedding, inputs)
        
        # 这里其实是按照处理顺序一步步的处理了数据，并没有使用sequential模型 
        
        # 开始搭建网络模型，使用从embedding中查找出的词向量
        # 作为输入，通过dropout层的数据形状不变，但是部分元素会变成0
        x = self.drop1(x, training=training)
        
        # 通过rnn层后，词向量的维度会变化成指定的神经元的个数
        x = self.rnn1(x)

        x = self.drop2(x, training=training)
        
        x = self.rnn2(x)

        x = self.drop3(x, training=training)
        
        x = self.rnn3(x)

        x = self.drop_fc(x, training=training)
        
        x = self.fc(x)

        x = self.out_linear(x)
        
        # 返回最后的输出值，即属于两个类别的概率
        return x

### 设置参数

In [22]:
params = {
  'vocab_path': './vocab/word.txt',
  'train_path': './data/train.txt',
  'test_path': './data/test.txt',
  'num_samples': 5000,
  'num_labels': 2,
  'batch_size': 64,
  'max_len': 300, # 句子的最大长度
  'rnn_units': 128,
  'dropout_rate': 0.2,
  'num_patience': 3, # 容忍次数
  'lr': 0.0001,
}

用来判断迭代何时停止

In [23]:
# 这里使用了函数注解，参数后面的冒号表示该参数希望传入的类型
def is_descending(history:list):
    
    history = history[-(params['num_patience']+1):]
    
    for i in range(1, len(history)):
        if history[i-1] <= history[i]:
            return False
        
    return True  

In [24]:
word2idx = {}

# 读入word2id，其实可以使用json格式保存到硬盘
# 但是之前使用的是txt文件
with open(params['vocab_path'], encoding='utf-8') as f:
    for i, line in enumerate(f):
        line = line.strip()
        word2idx[line] = i

# 添加两个参数
params['word2idx'] = word2idx
params['vocab_size'] = len(word2idx) + 1


# 这里的Model是刚才自定义的Model对象:
# class Model(tf.keras.Model):
model = Model(params)

# #设置输入的大小，或者fit时候也能自动找到
# model.build(input_shape=(None, None))

# return initial_learning_rate * decay_rate ^ (step / decay_steps)
# 相当于加了一个指数衰减函数
decay_lr = tf.optimizers.schedules.ExponentialDecay(params['lr'], 
                    decay_steps=1000, decay_rate=0.95)

optim = tf.optimizers.Adam(learning_rate=0.001)

global_step = 0

# 保存历史准确率
history_acc = []

best_acc = 0.0

t0 = time.time()
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)

In [25]:
# optim的lr属性保存了优化器的学习率，这个学习率可以是一个
# 标量，也可以是一个优化对象，具体查看API
optim.lr

<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.001>

In [26]:
# 可以自己定义终止训练的条件
while best_acc == 0 or best_acc < 0.8:
    
    
    # 从数据生成器中取数据训练模型，Dataset对象中每个元素都是一个二元组
    # 格式类似于:([1, 2, 3, 4, 5, ..., 100], 1)，第一个元素是句子，第二个元素是标签值
    for texts, labels in dataset(is_training=True, params=params):
        
        # 梯度带，监视所有在上下文中的变量
        # 然后可以通过调用.gradient()获得任何上下文中计算得出的张量的梯度
        with tf.GradientTape() as tape:
            
            # 由于继承了tf.keras.model对象，
            # 这里其实相当于调用model对象的call()方法
            # 拿到logits相当于完成了前向传播过程
            logits = model(texts, training=True)
            
            # 定义损失值
            loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
            
            # 取平均值
            loss = tf.reduce_mean(loss)
        
        # 下面进行反向传播，指定新的学习率
        optim.lr.assign(decay_lr(global_step))
        
        # trainable_variables属性保存了模型中所有的可训练变量
        grads = tape.gradient(loss, model.trainable_variables)
        

        # 利用刚才计算得到的梯度，使用优化器更新参数
        # 这里其实也可以调用optim.minimize()方法，请参考API
        optim.apply_gradients(zip(grads, model.trainable_variables))

        if global_step % 50 == 0:
            logger.info("Step {} | Loss: {:.4f} | Spent: {:.1f} secs | LR: {:.6f}".format(
                global_step, loss.numpy().item(), time.time()-t0, optim.lr.numpy().item()))
            t0 = time.time()
            
        global_step += 1

    m = tf.keras.metrics.Accuracy()
    
    # 从测试集中取数据
    for texts, labels in dataset(is_training=False, params=params):
        
        logits = model(texts, training=False)
        y_pred = tf.argmax(logits, axis=-1)
        
        # 计算准确率
        m.update_state(y_true=labels, y_pred=y_pred)
    
    # 返回结果
    acc = m.result().numpy()
    
    logger.info("Evaluation: Testing Accuracy: {:.3f}".format(acc))
    
    history_acc.append(acc)
  
    if acc > best_acc: best_acc = acc
        
    logger.info("Best Accuracy: {:.3f}".format(best_acc))
  
    if len(history_acc) > params['num_patience'] and is_descending(history_acc):
        logger.info("Testing Accuracy not improved over {} epochs, Early Stop".format(params['num_patience']))
        break

Reading ./data/train.txt
INFO:tensorflow:Step 0 | Loss: 0.6940 | Spent: 3.1 secs | LR: 0.000100
INFO:tensorflow:Step 50 | Loss: 0.6747 | Spent: 8.4 secs | LR: 0.000100
INFO:tensorflow:Step 100 | Loss: 0.6700 | Spent: 8.0 secs | LR: 0.000099
INFO:tensorflow:Step 150 | Loss: 0.6370 | Spent: 8.6 secs | LR: 0.000099
INFO:tensorflow:Step 200 | Loss: 0.6402 | Spent: 8.2 secs | LR: 0.000099
INFO:tensorflow:Step 250 | Loss: 0.8351 | Spent: 8.7 secs | LR: 0.000099
INFO:tensorflow:Step 300 | Loss: 0.6316 | Spent: 9.2 secs | LR: 0.000098
INFO:tensorflow:Step 350 | Loss: 0.5982 | Spent: 8.6 secs | LR: 0.000098
Reading ./data/test.txt
INFO:tensorflow:Evaluation: Testing Accuracy: 0.682
INFO:tensorflow:Best Accuracy: 0.682
Reading ./data/train.txt
INFO:tensorflow:Step 400 | Loss: 0.6197 | Spent: 32.5 secs | LR: 0.000098
INFO:tensorflow:Step 450 | Loss: 0.5912 | Spent: 7.0 secs | LR: 0.000098
INFO:tensorflow:Step 500 | Loss: 0.5484 | Spent: 6.7 secs | LR: 0.000097
INFO:tensorflow:Step 550 | Loss: 0.5